In [4]:
%pip install scipy --quiet
%pip install tenacity --quiet
%pip install tiktoken --quiet
%pip install termcolor --quiet
%pip install openai --quiet
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  
import dotenv
import os

import os

# Try to get the GITHUB_TOKEN from the system environment variables
github_token = os.environ["GITHUB_TOKEN"]
if not github_token:
    raise ValueError("GITHUB_TOKEN is not set in your system environment variables")

# Set the required OpenAI environment variables using the retrieved token
os.environ["OPENAI_API_KEY"] = github_token
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

print("Environment variables have been set successfully.")

GPT_MODEL = "gpt-4o-mini"

client = OpenAI()

KeyError: 'GITHUB_TOKEN'

In [2]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

In [3]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "What is the name of the album with the most tracks?"
}]

response = client.chat.completions.create(
    model='gpt-4o-mini', 
    messages=messages, 
    tools= tools, 
    tool_choice="auto"
)

# Append the message to messages list
response_message = response.choices[0].message 
messages.append(response_message)

print(response_message)

NameError: name 'client' is not defined